In [74]:
from os import getenv
from borsdata_sdk import BorsdataAPI
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.dpi'] = 200


api_key = '14e98818e51e42928578dd7e421f0e1d'
bors_api = BorsdataAPI(api_key)

In [75]:
'''report data = {'year': 2018, 'period': 5, 'revenues': 27772.0, 'gross_Income': 7049.0, 
'operating_Income': 1956.0, 'profit_Before_Tax': 1829.0, 'profit_To_Equity_Holders': 1322.0, 
'earnings_Per_Share': 5.2103, 'number_Of_Shares': 253.728, 'dividend': 1.85, 'intangible_Assets': 1999.0,
'tangible_Assets': 5468.0, 'financial_Assets': 91.0, 'non_Current_Assets': 7558.0, 'cash_And_Equivalents': 737.0, 
'current_Assets': 10477.0, 'total_Assets': 18035.0, 'total_Equity': 8996.0, 'non_Current_Liabilities': 3681.0, 
'current_Liabilities': 5358.0, 'total_Liabilities_And_Equity': 18035.0, 'net_Debt': 2667.0, 
'cash_Flow_From_Operating_Activities': 1090.0, 'cash_Flow_From_Investing_Activities': -723.0, 
'cash_Flow_From_Financing_Activities': -123.0, 'cash_Flow_For_The_Year': 244.0, 'free_Cash_Flow': 367.0, 
'stock_Price_Average': 134.95, 'stock_Price_High': 156.9, 'stock_Price_Low': 113.0, 
'report_Start_Date': '2018-01-01T00:00:00', 'report_End_Date': '2018-12-31T00:00:00', 
'broken_Fiscal_Year': False}'''

def load_report_data():
    instruments = bors_api.get_instruments()
    reports = {}
    idents = []
    tickers = []
    for instrument in instruments:
        stock = instrument.ticker
        ident = instrument.insId
        reports[ident] = bors_api.get_instrument_reports(ident, 'year')
        idents.append(ident)
        tickers.append(stock)
    return idents, tickers, reports

# This is the loading of the reporting data - only to be updated quarterly

In [76]:
'''This is not needed to be updated on a regular basis, only once in a while'''
idents, tickers, reports = load_report_data()

In [77]:
print(reports[2][0].dividend)

1.85


In [91]:
#columns = ['id', 'ticker', 'close', 'dividend', 'earnings', 'growth_rev_5y', 'growth_ear_5y', 'growth_div_5y', 'equity_ration']
analysis_data = pd.DataFrame()


analysis_data['id'] = idents
analysis_data['ticker'] = tickers
analysis_data['close'] = None
analysis_data['dividend'] = None
analysis_data['earnings'] = None
analysis_data['growth_rev_5y'] = None
analysis_data['growth_ear_5y'] = None
analysis_data['growth_div_5y'] = None
analysis_data['equity_ratio'] = None
analysis_data['p/e'] = None
analysis_data['profit_return'] = None
analysis_data['direct_return'] = None
analysis_data['country'] = None

for i, row in analysis_data.iterrows():
    row['close'] = prices[i].c
    report = reports[int(row['id'])]
    
    if int(row['id']) in reports.keys() and len(reports[int(row['id'])])>0:
        row['dividend'] = report[0].dividend
        row['earnings'] = report[0].earnings_Per_Share
        if len(report) > 4 and float(report[4].revenues)>0 and (float(report[0].revenues)-float(report[4].revenues))>0:
            row['growth_rev_5y'] = np.power(((float(report[0].revenues)-float(report[4].revenues))/float(report[4].revenues)),0.2)
        if len(report) > 4 and float(report[4].earnings_Per_Share)>0 and (float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))>0:
            row['growth_ear_5y'] = np.power(((float(report[0].earnings_Per_Share)-float(report[4].earnings_Per_Share))/float(report[4].earnings_Per_Share)),0.2)
        if len(report) > 4 and float(report[4].dividend)>0 and (float(report[0].dividend)-float(report[4].dividend))>0:
            row['growth_div_5y'] = np.power(((float(report[0].dividend)-float(report[4].dividend))/float(report[4].dividend)),0.2)
        if float(report[0].total_Assets)>0:
            row['equity_ratio']  = float(report[0].total_Equity)/float(report[0].total_Assets)
    analysis_data.iloc[i] = row


# This segment are the buy candidates - to be updated daily

In [176]:
'''This should be updated daily'''
prices = bors_api.get_instrument_stock_price_last()
pricedict = {}
countrydict = {}

for price in prices:
    pricedict[price.i] = price.c 

for instrument in instruments:
    countrydict[instrument.insId] = instrument.countryId
    
for i, row in analysis_data.iterrows():
    index = row['id']
    row['close'] = pricedict[index]
    row['p/e'] = float(pricedict[index])/float(row['earnings']) if row['earnings'] is not None and float(row['earnings'])!=0 else None
    row['direct_return'] = float(row['dividend'])/float(pricedict[index]) if row['dividend'] is not None and float(row['dividend'])!=0 else None
    row['profit_return'] = 1/row['p/e'] if row['p/e'] is not None else None
    row['country'] = int(countrydict[index])
    analysis_data.iloc[i] = row

In [177]:
n = 20
div_req = 0.02
prof_g = 0.1

filtered_results=analysis_data[(analysis_data.direct_return > div_req) 
                               & (analysis_data.profit_return>analysis_data.direct_return) 
                               & (analysis_data.growth_ear_5y > prof_g)
                               & (analysis_data.country == 1)].sort_values(['profit_return'], ascending=False)
tickers = filtered_results.ticker[0:n].to_dict()
print(filtered_results.head(n))

        id    ticker   close dividend earnings growth_rev_5y growth_ear_5y  \
1376  1627    SAGA D    37.2        2       10       1.01612       1.26493   
98     121    KLOV A   16.35     0.46   3.9983      0.780245       1.17112   
1368  1619    KLOV B   16.38     0.46   3.9983      0.780245       1.17112   
276    349      TETY    72.3        8  15.8981      0.790319      0.907211   
1357  1608    CORE B    18.8     0.45   3.2421      0.888574       1.56334   
43      57    CORE A    18.9     0.45   3.2421      0.888574       1.56334   
22      32    BETS B   47.15     3.89   7.7897      0.952911      0.825222   
298    383      G5EN   87.75      2.5  14.4504       1.47417       1.77587   
290    374      HEMF    91.1      2.4  14.7532      0.892527      0.928412   
239    300    GHUS B    4.17     0.15   0.6619        0.9393       1.43223   
177    217    SWED A   122.3     14.2  19.0027      0.659888      0.770979   
154    191      RROS    9.59      0.7   1.4484      0.868347    

In [178]:
filtered_results_div=analysis_data[(analysis_data.direct_return > div_req) 
                               & (analysis_data.profit_return>analysis_data.direct_return) 
                               & (analysis_data.growth_ear_5y > prof_g)
                               & (analysis_data.country == 1)].sort_values(['direct_return'], ascending=False)
tickers_div = filtered_results_div.ticker[0:n].to_dict()
print(filtered_results_div.head(n))

        id    ticker   close dividend earnings growth_rev_5y growth_ear_5y  \
131    159    NDA SE    57.8    7.176   7.9127          None      0.492537   
190    231  KIND SDB   49.49   5.9024   6.8927       1.12314       0.80636   
177    217    SWED A   122.3     14.2  19.0027      0.659888      0.770979   
276    349      TETY    72.3        8  15.8981      0.790319      0.907211   
95     118      KAHL   19.57        2   2.9159      0.324249      0.930157   
203    245      DEDI    40.9        4   4.2186      0.873185      0.798554   
125    153     NET B    23.5     2.25   2.4038       1.01782       1.06443   
281    357      WISE    40.9      3.5   4.7767       0.93506       1.22888   
22      32    BETS B   47.15     3.89   7.7897      0.952911      0.825222   
28      38      BORG    25.3        2   2.3909      0.794419      0.746112   
160    199     SEB A   82.42      6.5  10.6706          None      0.734874   
194    236    VOLV B  128.55       10  12.2645      0.824616    

In [179]:
#countries = bors_api.get_markets()
candidates = []
for i, result in tickers.items():
    if i in tickers_div.keys():
        candidates.append(i)
        print(result)
        print(instruments[i].__dict__)
        
print(analysis_data.iloc[candidates])

TETY
{'insId': 349, 'name': 'Tethys Oil', 'urlName': 'tethys-oil', 'instrument': 0, 'isin': 'SE0012455202', 'ticker': 'TETY', 'yahoo': 'TETY.ST', 'sectorId': 3, 'marketId': 2, 'branchId': 2, 'countryId': 1, 'listingDate': '2004-04-06T00:00:00'}
BETS B
{'insId': 32, 'name': 'Betsson', 'urlName': 'betsson', 'instrument': 0, 'isin': 'SE0012454379', 'ticker': 'BETS B', 'yahoo': 'BETS-B.ST', 'sectorId': 8, 'marketId': 1, 'branchId': 54, 'countryId': 1, 'listingDate': '1996-03-26T00:00:00'}
SWED A
{'insId': 217, 'name': 'Swedbank', 'urlName': 'swedbank', 'instrument': 0, 'isin': 'SE0000242455', 'ticker': 'SWED A', 'yahoo': 'SWED-A.ST', 'sectorId': 1, 'marketId': 1, 'branchId': 68, 'countryId': 1, 'listingDate': '1995-06-09T00:00:00'}
RROS
{'insId': 191, 'name': 'Rottneros', 'urlName': 'rottneros', 'instrument': 0, 'isin': 'SE0000112252', 'ticker': 'RROS', 'yahoo': 'RROS.ST', 'sectorId': 7, 'marketId': 3, 'branchId': 21, 'countryId': 1, 'listingDate': '1987-11-20T00:00:00'}
KAHL
{'insId': 118

# This segment are the potential sales candidates - to be updated daily

In [196]:
filtered_results_neg=analysis_data[(analysis_data.country == 1)].sort_values(['profit_return'], ascending=True)
tickers_neg = filtered_results_neg.ticker[0:n].to_dict()

for i, result in tickers_neg.items():
    print(i)
    print(instruments[i].name)
print(filtered_results_neg.head(n))

263
Saxlund
613
Prime Living
340
Papilly
222
Confidence
338
Nexstim
346
Italeaf
455
Lifeassays
392
Red Reserve
57
Eniro
1225
Matra Petroleum
235
Wifog
1183
Curando Nordic
618
TargetEveryOne
464
Metallvärden
614
SavoSolar
432
Eurocine Vaccines
1149
WilLak
479
Intuitive Aerial
1302
Artificial Solutions
1226
Mantex
        id    ticker  close dividend earnings growth_rev_5y growth_ear_5y  \
263    331      SAXG  0.112        0  -3.8549        0.9297          None   
613    760     PRIME   1.18        0 -38.9553       1.21557          None   
340    432      PAPI  0.062        0    -1.05       2.33014          None   
222    277      CONF   1.15        0 -14.3133      0.828015          None   
338    430     NXTMS    2.1        0 -22.4383          None          None   
346    439  ITAL SDB   0.13        0  -1.1469          None          None   
455    573    LIFE B  0.778        0  -6.6597      0.824115          None   
392    496      REDR  0.212        0  -1.4535       1.20401          N